In [ ]:
!apt-get install python3.5
#!pip3 install 'torch==0.3.1'
!pip3 install 'torch==0.4.0'
!pip3 install tensorboardX

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'libpython3.5-minimal' for regex 'python3.5'
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
     |████████████████████████████████| 484.0MB 36kB/s 
ERROR: torchvision 0.7.0+cu101 has requirement torch==1.6.0, but you'll have torch 0.4.0 which is incompatible.
ERROR: fastai 1.0.61 has requirement torch>=1.0.0, but you'll have torch 0.4.0 which is incompatible.
  Found existing installation: torch 1.6.0+cu101
    Uninstalling torch-1.6.0+cu101:
      Successfully uninstalled torch-1.6.0+cu101
     |████████████████████████████████| 317kB 4.6MB/s 


In [ ]:
from google.colab import drive
import os
drive.mount('/content/gdrive/')
path = './gdrive/My Drive/Ego-rnn'
os.chdir(path)
cwd = os.getcwd()
print('Current dir: ' + cwd)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive/
Current dir: /content/gdrive/My Drive/Ego-rnn


In [ ]:
from __future__ import print_function, division
from objectAttentionModelConvLSTM_Variation import *
from spatial_transforms import (Compose, ToTensor, CenterCrop, Scale, Normalize, MultiScaleCornerCrop,
                                RandomHorizontalFlip)
from tensorboardX import SummaryWriter
#import makeDatasetRGB as RGB
from makeDatasetMMAPS_Variation  import *
import argparse
import sys


def main_run(dataset, stage, train_data_dir, val_data_dir, stage1_dict, out_dir, seqLen, trainBatchSize,
             valBatchSize, numEpochs, lr1, decay_factor, decay_step, memSize, regression):


    if dataset == 'gtea61':
        num_classes = 61
    elif dataset == 'gtea71':
      num_classes = 71
    elif dataset == 'gtea_gaze':
        num_classes = 44
    elif dataset == 'egtea':
        num_classes = 106
    else:
        print('Dataset not found')
        sys.exit()
		

    model_folder = os.path.join('./', out_dir, dataset, 'rgb_ms', 'stage'+str(stage))  # Dir for saving models and log files
    # Create the dir
    if os.path.exists(model_folder):
        print('Directory {} exists!'.format(model_folder))
        sys.exit()
        
    os.makedirs(model_folder)

    # Log files - sono log file, direi che per ora possiamo trascurarli e magari guardarli quando runniamo!
    writer = SummaryWriter(model_folder)
    train_log_loss = open((model_folder + '/train_log_loss.txt'), 'w')
    train_log_acc = open((model_folder + '/train_log_acc.txt'), 'w')
    val_log_loss = open((model_folder + '/val_log_loss.txt'), 'w')
    val_log_acc = open((model_folder + '/val_log_acc.txt'), 'w')

    train_log_loss_ms= open((model_folder + '/train_log_loss_ms.txt'), 'w')
    val_log_loss_ms = open((model_folder + '/val_log_loss_ms.txt'), 'w')
    train_log_acc_ms= open((model_folder + '/train_log_acc_ms.txt'), 'w')
    val_log_acc_ms = open((model_folder + '/val_log_acc_ms.txt'), 'w')

    train_log_loss_ot = open((model_folder + '/train_log_loss_ot.txt'), 'w')
    val_log_loss_ot = open((model_folder + '/val_log_loss_ot.txt'), 'w')
    train_log_acc_ot = open((model_folder + '/train_log_acc_ot.txt'), 'w')
    val_log_acc_ot = open((model_folder + '/val_log_acc_ot.txt'), 'w')

    # Data loader
    #normalize = Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    #spatial_transform = Compose([Scale(256), RandomHorizontalFlip(), MultiScaleCornerCrop([1, 0.875, 0.75, 0.65625], 224),
    #                             ToTensor(), normalize])
    spatial_transform = Compose([Scale(256), RandomHorizontalFlip(), MultiScaleCornerCrop([1, 0.875, 0.75, 0.65625], 224)])

    vid_seq_train = makeDataset(train_data_dir, spatial_transform=spatial_transform, seqLen=seqLen, fmt='.png', regression=regression)

    train_loader = torch.utils.data.DataLoader(vid_seq_train, batch_size=trainBatchSize,
                            shuffle=True, num_workers=4, pin_memory=True)
    
    if val_data_dir is not None:
        vid_seq_val = makeDataset(val_data_dir, train=False,
                                   spatial_transform=Compose([Scale(256), CenterCrop(224)]), #ToTensor(), normalize]),
                                   seqLen=seqLen, fmt='.png', regression=regression)
        
        val_loader = torch.utils.data.DataLoader(vid_seq_val, batch_size=valBatchSize,
                                shuffle=False, num_workers=2, pin_memory=True)
        valInstances = vid_seq_val.__len__()


    trainInstances = vid_seq_train.__len__()
    print(f'Train instances: {trainInstances}')
	
    train_params = []
    if stage == 1:

        model = attentionModel(num_classes=num_classes, mem_size=memSize, regression=regression, seqLen=seqLen)
        model.train(False)
        for params in model.parameters():
            params.requires_grad = False
    else:
        model = attentionModel(num_classes=num_classes, mem_size=memSize, regression=regression, seqLen=seqLen)
        model.load_state_dict(torch.load(stage1_dict))
        model.train(False)
        for params in model.parameters():
            params.requires_grad = False
        #
        for params in model.resNet.layer4[0].conv1.parameters():
            params.requires_grad = True
            train_params += [params]

        for params in model.resNet.layer4[0].conv2.parameters():
            params.requires_grad = True
            train_params += [params]

        for params in model.resNet.layer4[1].conv1.parameters():
            params.requires_grad = True
            train_params += [params]

        for params in model.resNet.layer4[1].conv2.parameters():
            params.requires_grad = True
            train_params += [params]

        for params in model.resNet.layer4[2].conv1.parameters():
            params.requires_grad = True
            train_params += [params]
        #
        for params in model.resNet.layer4[2].conv2.parameters():
            params.requires_grad = True
            train_params += [params]
        #
        for params in model.resNet.fc.parameters():
            params.requires_grad = True
            train_params += [params]

        #if MSTask == True :
        for params in model.convMS.parameters():
            params.requires_grad = True
            train_params += [params]
        for params in model.MSfc.parameters():
            params.requires_grad = True
            train_params += [params]
        if regression == False:
            for params in model.m.parameters():
                params.requires_grad = True
                train_params += [params]
            for params in model.orderPredictionNetwork.parameters():
                params.requires_grad = True
                train_params += [params]

        model.resNet.layer4[0].conv1.train(True)
        model.resNet.layer4[0].conv2.train(True)
        model.resNet.layer4[1].conv1.train(True)
        model.resNet.layer4[1].conv2.train(True)
        model.resNet.layer4[2].conv1.train(True)
        model.resNet.layer4[2].conv2.train(True)
        model.resNet.fc.train(True)

        model.convMS.train(True)
        model.MSfc.train(True)
        if regression == False:
            model.m.train(True)
            model.orderPredictionNetwork.train(True)

    for params in model.lstm_cell.parameters():
        params.requires_grad = True
        train_params += [params]

    for params in model.classifier.parameters():
        params.requires_grad = True
        train_params += [params]


    model.lstm_cell.train(True)

    model.classifier.train(True)
    model.cuda()

    loss_fn = nn.CrossEntropyLoss()

    loss_pretext = nn.MSELoss()

    optimizer_fn = torch.optim.Adam(train_params, lr=lr1, weight_decay=4e-5, eps=1e-4)

    optim_scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer_fn, milestones=decay_step,
                                                           gamma=decay_factor)

	  #TRAINING
    train_iter = 0
    min_accuracy = 0

    for epoch in range(numEpochs):
        optim_scheduler.step()
        epoch_loss = 0
        epoch_loss_ms = 0
        epoch_loss_ot = 0
        numCorrTrain = 0
        numCorrTrain_ms = 0
        numCorrTrain_ot = 0
        trainSamples = 0
        iterPerEpoch = 0
        model.lstm_cell.train(True)
        model.classifier.train(True)
        writer.add_scalar('lr', optimizer_fn.param_groups[0]['lr'], epoch+1)
        if stage == 2:
            model.resNet.layer4[0].conv1.train(True)
            model.resNet.layer4[0].conv2.train(True)
            model.resNet.layer4[1].conv1.train(True)
            model.resNet.layer4[1].conv2.train(True)
            model.resNet.layer4[2].conv1.train(True)
            model.resNet.layer4[2].conv2.train(True)
            model.resNet.fc.train(True)
            model.resNet.fc.train(True)
            #if MSTask == True:
            model.convMS.train(True)
            model.MSfc.train(True)
            if regression == False:
                model.m.train(True)
                model.orderPredictionNetwork.train(True)
        for i, (inputs, inputs_maps, targets, idxPerm) in enumerate(train_loader):
            train_iter += 1
            iterPerEpoch += 1
            optimizer_fn.zero_grad()
            #labelMapsVariable = Variable(labels_maps.cuda())
            inputVariable = Variable(inputs.permute(1, 0, 2, 3, 4).cuda())
            labelVariable = Variable(targets.cuda())
            orderVariable = Variable(torch.squeeze(idxPerm).cuda())
            trainSamples += inputs.size(0)
            output_label, _, ms_labels, order_pred = model(inputVariable, regression, orderVariable)
            loss = loss_fn(output_label, labelVariable)
            if stage == 2:
                loss.backward(retain_graph=True)
            else:
                loss.backward()
            if regression == False:
                inputMapsVariable = Variable(inputs_maps.permute(1, 0, 2, 3, 4).type(torch.LongTensor).cuda())
                ms_labels = ms_labels.view(-1, 2)
            else:
                inputMapsVariable = Variable(inputs_maps.permute(1, 0, 2, 3, 4).cuda())
                ms_labels = ms_labels.view(-1)
            inputMapsVariable = inputMapsVariable.contiguous().view(-1)
            if stage == 2:
                if regression == False:
                    loss_ms = loss_fn(ms_labels, inputMapsVariable)
                    _, predicted = torch.max(ms_labels.data, 1)
                    #numCorrTrain_ms += (predicted == targets.cuda()).sum()
                    #numCorrTrain_ms += torch.sum(predicted == inputMapsVariable.data).data.item()
                    numCorrTrain_ms += torch.sum(predicted == inputMapsVariable.data)
                    loss_ot = loss_fn(order_pred, orderVariable)
                    _, predicted = torch.max(order_pred.data, 1)
                    numCorrTrain_ot += torch.sum(predicted == orderVariable.data).data.item()
                    final_loss = loss_ms + loss_ot
                    epoch_loss_ms += loss_ms.data[0]
                    epoch_loss_ot += loss_ot.data[0]
                else:
                    final_loss = loss_pretext(ms_labels, inputMapsVariable)
                    epoch_loss_ms += loss_ms.data[0]
                final_loss.backward()
            optimizer_fn.step()
            _, predicted = torch.max(output_label.data, 1)
            #numCorrTrain += (predicted == targets.cuda()).sum()
            numCorrTrain += torch.sum(predicted == labelVariable.data).data.item()
            epoch_loss += loss.data[0]
        avg_loss = epoch_loss/iterPerEpoch
        if stage == 2:
            #if MSTask == True:
            avg_loss_ms = epoch_loss_ms/iterPerEpoch
            if regression == False:
              avg_loss_ot = epoch_loss_ot/iterPerEpoch
              train_log_loss_ot.write('Train Loss Order Task after {} epochs = {}\n'.format(epoch + 1, avg_loss_ot))
              trainAccuracy_ot = (numCorrTrain_ot / trainSamples) * 100
              train_log_acc_ot.write('Train Accuracy Order Task after {} epochs = {}\n'.format(epoch + 1, trainAccuracy_ot))
            train_log_loss_ms.write('Train Loss MS after {} epochs = {}\n'.format(epoch + 1, avg_loss_ms))
            trainAccuracy_ms = (numCorrTrain_ms / trainSamples) * 100
            train_log_acc_ms.write('Train Accuracy MS after {} epochs = {}\n'.format(epoch + 1, trainAccuracy_ms))
        trainAccuracy = (numCorrTrain / trainSamples) * 100
        #if stage == 2 and MSTask == True:
        if stage == 2:
            if regression == True:
                print('Train: Epoch = {} | Loss = {} | MS_Loss = {} | Accuracy = {}'.format(epoch+1, avg_loss, avg_loss_ms, trainAccuracy))
            else:
                print('Train: Epoch = {} | Loss = {} | MS_Loss = {} | OrderTask Loss = {} | Accuracy = {}'.format(epoch+1, avg_loss, avg_loss_ms, avg_loss_ot, trainAccuracy))
        else:
            print('Train: Epoch = {} | Loss = {} | Accuracy = {}'.format(epoch+1, avg_loss, trainAccuracy))

        writer.add_scalar('train/epoch_loss', avg_loss, epoch+1)
        writer.add_scalar('train/accuracy', trainAccuracy, epoch+1)

        if stage == 2:
            writer.add_scalar('train/epoch_MS_loss', avg_loss_ms, epoch+1)
        if val_data_dir is not None:
            if (epoch+1) % 1 == 0: #???
                model.train(False)
                val_loss_epoch = 0
                val_loss_epoch_ms = 0
                val_loss_epoch_ot = 0
                val_iter = 0
                val_samples = 0
                numCorr = 0
                numCorr_ms = 0
                numCorr_ot = 0
                for j, (inputs, inputs_maps, targets, idxPerm) in enumerate(val_loader):
                    val_iter += 1
                    val_samples += inputs.size(0)
                    #labelMapsVariable = Variable(labels_maps.cuda())
                    inputVariable = Variable(inputs.permute(1, 0, 2, 3, 4).cuda(), volatile=True)
                    labelVariable = Variable(targets.cuda(async=True), volatile=True)
                    orderVariable = Variable(torch.squeeze(idxPerm).cuda(), volatile = True)
                    output_label, _, ms_labels, order_pred = model(inputVariable, regression, orderVariable)
                    #output_label, _, ms_labels = model(inputVariable, regression)
                    val_loss = loss_fn(output_label, labelVariable) 
                    val_loss_epoch += val_loss.data[0]
                    if regression == False:
                        inputMapsVariable = Variable(inputs_maps.permute(1, 0, 2, 3, 4).type(torch.LongTensor).cuda(), volatile=True)
                        ms_labels = ms_labels.view(-1,2)
                    else:
                        inputMapsVariable = Variable(inputs_maps.permute(1, 0, 2, 3, 4).cuda(), volatile=True)
                        ms_labels = ms_labels.view(-1)
                    inputMapsVariable = inputMapsVariable.contiguous().view(-1)
                    if stage == 2:
                        if regression == True:
                            val_loss_ms = loss_pretext(ms_labels, inputMapsVariable)
                        else:
                            val_loss_ms = loss_fn(ms_labels, inputMapsVariable)
                            _, predicted = torch.max(ms_labels.data, 1)
                            numCorr_ms += torch.sum(predicted == inputMapsVariable.data)
                            val_loss_ot = loss_fn(order_pred, orderVariable)
                            _, predicted = torch.max(order_pred.data, 1)
                            numCorr_ot += torch.sum(predicted == orderVariable.data).data.item()
                        val_loss_epoch_ms += val_loss_ms.data[0]
                        val_loss_epoch_ot += val_loss_ot.data[0]
                    _, predicted = torch.max(output_label.data, 1)
                    #numCorr += (predicted == targets.cuda()).sum()
                    numCorr += torch.sum(predicted == labelVariable.data).data.item() 
                    '''else:
                        val_loss_ms = loss_fn(order_pred, orderVariable)
                        _, predicted = torch.max(order_pred.data, 1)
                        numCorr_ms += torch.sum(predicted == orderVariable.data).data.item()                           
                        #numCorr_ms += torch.sum(predicted == inputMapsVariable.data).data.item()'''
                val_accuracy = (numCorr / val_samples) * 100
                avg_val_loss = val_loss_epoch / val_iter
                if stage == 2:
                    avg_val_loss_ms = val_loss_epoch_ms / val_iter
                    val_log_loss_ms.write('Val Loss MS after {} epochs = {}\n'.format(epoch + 1, avg_val_loss_ms))
                    avg_val_loss_ot = val_loss_epoch_ot / val_iter
                    val_log_loss_ot.write('Val Loss Order Task after {} epochs = {}\n'.format(epoch + 1, avg_val_loss_ot))
                    if regression == False:
                        val_accuracy_ms = (numCorr_ms / val_samples) * 100
                        val_log_acc_ms.write('Val Accuracy MS after {} epochs = {}\n'.format(epoch + 1, val_accuracy_ms))
                        val_accuracy_ot = (numCorr_ot / val_samples) * 100
                        val_log_acc_ot.write('Val Accuracy Order Task after {} epochs = {}\n'.format(epoch + 1, val_accuracy_ot))
                if stage == 2:
                    print('Val: Epoch = {} | Loss = {} | MS_Loss = {} | Order Task Loss = {} | Accuracy = {}'.format(epoch + 1, avg_val_loss, avg_val_loss_ms, avg_val_loss_ot, val_accuracy))
                else:
                    print('Val: Epoch = {} | Loss = {} | Accuracy = {}'.format(epoch + 1, avg_val_loss, val_accuracy))
                writer.add_scalar('val/epoch_loss', avg_val_loss, epoch + 1)
                writer.add_scalar('val/accuracy', val_accuracy, epoch + 1)
                if stage == 2:
                    writer.add_scalar('val/epoch_MS_loss', avg_val_loss_ms, epoch + 1)
                    writer.add_scalar('val/epoch_OT_loss', avg_val_loss_ot, epoch + 1)
                val_log_loss.write('Val Loss after {} epochs = {}\n'.format(epoch + 1, avg_val_loss))
                val_log_acc.write('Val Accuracy after {} epochs = {}%\n'.format(epoch + 1, val_accuracy))
				        #CHECK!!!
                if val_accuracy > min_accuracy:
                    save_path_model = (model_folder + '/model_rgb_state_dict.pth')
                    torch.save(model.state_dict(), save_path_model)
                    min_accuracy = val_accuracy
            else:
                if (epoch+1) % 10 == 0:
                    save_path_model = (model_folder + '/model_rgb_state_dict_epoch' + str(epoch+1) + '.pth')
                    torch.save(model.state_dict(), save_path_model)

    train_log_loss.close()
    train_log_acc.close()
    val_log_acc.close()
    val_log_loss.close()
    train_log_loss_ms.close()
    val_log_loss_ms.close()
    train_log_acc_ms.close()
    val_log_acc_ms.close()
    train_log_loss_ot.close()
    val_log_loss_ot.close()
    train_log_acc_ot.close()
    val_log_acc_ot.close()
    writer.export_scalars_to_json(model_folder + "/all_scalars.json")
    writer.close()


def __main__():
    
    parser = argparse.ArgumentParser()

    #STAGE 1
    
    parser.add_argument('--dataset', type=str, default='gtea61', help='Dataset')
    parser.add_argument('--stage', type=int, default=1, help='Training stage')
    parser.add_argument('--trainDatasetDir', type=str, default='./GTEA61', help='Train set directory')
    parser.add_argument('--valDatasetDir', type=str, default='./GTEA61', help='Val set directory')
    parser.add_argument('--outDir', type=str, default='Variation_16frames', help='Directory to save results')
    parser.add_argument('--stage1Dict', type=str, default='./Variation_16frames/gtea61/rgb/stage1/model_rgb_state_dict.pth', help='Stage 1 model path')
    parser.add_argument('--seqLen', type=int, default=16, help='Length of sequence')
    parser.add_argument('--trainBatchSize', type=int, default=32, help='Training batch size')
    parser.add_argument('--valBatchSize', type=int, default=64, help='Validation batch size')
    parser.add_argument('--numEpochs', type=int, default=300, help='Number of epochs')
    parser.add_argument('--lr', type=float, default=1e-3, help='Learning rate')
    parser.add_argument('--stepSize', type=float, default=[25, 75, 150], nargs="+", help='Learning rate decay step')
    parser.add_argument('--decayRate', type=float, default=0.1, help='Learning rate decay rate')
    parser.add_argument('--memSize', type=int, default=512, help='ConvLSTM hidden state size')
    parser.add_argument('--regressionFlag', type=int, default=False, help='Flag for regression in self-supervised task')

    #STAGE 2
    '''
    parser.add_argument('--dataset', type=str, default='gtea61', help='Dataset')
    parser.add_argument('--stage', type=int, default=2, help='Training stage')
    parser.add_argument('--trainDatasetDir', type=str, default='./GTEA61',
                        help='Train set directory')
    parser.add_argument('--valDatasetDir', type=str, default='./GTEA61',
                        help='Val set directory')
    parser.add_argument('--outDir', type=str, default='Variation_16frames', help='Directory to save results')
    parser.add_argument('--stage1Dict', type=str, default='./Variation_16frames/gtea61/rgb_ms/stage1/model_rgb_state_dict.pth',
                        help='Stage 1 model path')
    parser.add_argument('--seqLen', type=int, default=16, help='Length of sequence')
    parser.add_argument('--trainBatchSize', type=int, default=32, help='Training batch size')
    parser.add_argument('--valBatchSize', type=int, default=64, help='Validation batch size')
    parser.add_argument('--numEpochs', type=int, default=150, help='Number of epochs')
    parser.add_argument('--lr', type=float, default=1e-4, help='Learning rate')
    parser.add_argument('--stepSize', type=float, default=[25, 75], nargs="+", help='Learning rate decay step')
    parser.add_argument('--decayRate', type=float, default=0.1, help='Learning rate decay rate')
    parser.add_argument('--memSize', type=int, default=512, help='ConvLSTM hidden state size')
    parser.add_argument('--regressionFlag', type=bool, default=False, help='Flag for regression in self-supervised task')
    '''

    #args = parser.parse_args()
    args, unknown = parser.parse_known_args()

    dataset = args.dataset
    stage = args.stage
    trainDatasetDir = args.trainDatasetDir
    valDatasetDir = args.valDatasetDir
    outDir = args.outDir
    stage1Dict = args.stage1Dict
    seqLen = args.seqLen
    trainBatchSize = args.trainBatchSize
    valBatchSize = args.valBatchSize
    numEpochs = args.numEpochs
    lr1 = args.lr
    stepSize = args.stepSize
    decayRate = args.decayRate
    memSize = args.memSize
    regression = args.regressionFlag
    #MSTask = args.MSflag

    main_run(dataset, stage, trainDatasetDir, valDatasetDir, stage1Dict, outDir, seqLen, trainBatchSize, valBatchSize, numEpochs, lr1, decayRate, stepSize, memSize, regression)

__main__()


Train instances: 341


Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /root/.torch/models/resnet34-333f7ec4.pth
100%|██████████| 87306240/87306240 [00:02<00:00, 42653981.71it/s]
/content/gdrive/My Drive/Ego-rnn/MyConvLSTMCell.py:31: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  torch.nn.init.xavier_normal(self.conv_i_xx.weight)
/content/gdrive/My Drive/Ego-rnn/MyConvLSTMCell.py:32: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  torch.nn.init.constant(self.conv_i_xx.bias, 0)
/content/gdrive/My Drive/Ego-rnn/MyConvLSTMCell.py:33: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  torch.nn.init.xavier_normal(self.conv_i_hh.weight)
/content/gdrive/My Drive/Ego-rnn/MyConvLSTMCell.py:35: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  torch.nn.init.xavier_normal(self.conv_f_xx.weight)
/content/gdrive/My Drive/Ego-rnn/MyConvLST

Train: Epoch = 1 | Loss = 4.102425575256348 | Accuracy = 4.398826979472141


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:329: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:330: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:331: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:335: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:338: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.


Val: Epoch = 1 | Loss = 4.069135665893555 | Accuracy = 6.0344827586206895
Train: Epoch = 2 | Loss = 4.057037353515625 | Accuracy = 6.158357771260997
Val: Epoch = 2 | Loss = 3.9569907188415527 | Accuracy = 3.4482758620689653
Train: Epoch = 3 | Loss = 4.0105814933776855 | Accuracy = 4.105571847507331
Val: Epoch = 3 | Loss = 3.9280314445495605 | Accuracy = 6.0344827586206895
Train: Epoch = 4 | Loss = 4.018466949462891 | Accuracy = 5.865102639296188
Val: Epoch = 4 | Loss = 3.9678850173950195 | Accuracy = 6.896551724137931
Train: Epoch = 5 | Loss = 3.9893248081207275 | Accuracy = 6.451612903225806
Val: Epoch = 5 | Loss = 3.9039340019226074 | Accuracy = 6.0344827586206895
Train: Epoch = 6 | Loss = 3.8906803131103516 | Accuracy = 7.331378299120235
Val: Epoch = 6 | Loss = 3.8196234703063965 | Accuracy = 6.0344827586206895
Train: Epoch = 7 | Loss = 3.8744239807128906 | Accuracy = 8.504398826979472
Val: Epoch = 7 | Loss = 3.915343761444092 | Accuracy = 6.0344827586206895
Train: Epoch = 8 | Loss 

In [ ]:
from google.colab import drive
drive.flush_and_unmount()
print('All changes made in this colab session should now be visible in Drive.')

All changes made in this colab session should now be visible in Drive.
